In [1]:
import yfinance as yf
import talib
import pandas as pd
import numpy as np
import vectorbt as vbt 
from vectorbt.portfolio.enums import OrderSide
import torch

# data preprocess
target_stock = 1213
threshold = 0
stock_df = torch.load(f'dataset/torch-data/raw_test_data.pt')
stock_df = pd.DataFrame(stock_df)
stock_df = stock_df[stock_df['股票代號'] == target_stock].reset_index(drop = True)
five_day_min_low = talib.MIN(stock_df['low'], timeperiod=5) # 五天最低價當作long strategy的停損點
five_day_max_high = talib.MAX(stock_df['high'], timeperiod=5) # 五天最高價當作short strategy的停損點
stock_df['five_day_min_low'] = five_day_min_low
stock_df['five_day_max_high'] = five_day_max_high
stock_df = stock_df[5:]
predicted_price_df = pd.read_csv(f'predicted_result/{str(target_stock)} 2023 predicted prices.csv', encoding='utf-8', encoding_errors = 'ignore')
stock_df['日期'] = pd.to_datetime(stock_df['日期'])
stock_df.index = stock_df['日期']
stock_df['predicted_price'] = predicted_price_df['predicted_price'].values
stock_df['true_price'] = predicted_price_df['true_price'].values

# long short condition
stock_df['predicted_price_up'] = (stock_df['predicted_price'] > stock_df['predicted_price'].shift(1)).astype(int)
stock_df['actual_price_up'] = (stock_df['close'] > stock_df['close'].shift(1)).astype(int)
stock_df['long_condition'] = (stock_df['true_price'].shift(1)<=stock_df['predicted_price']) & ((stock_df['predicted_price']-stock_df['true_price'].shift(1))/stock_df['true_price'].shift(1) >= threshold)
stock_df['short_condition'] = (stock_df['true_price'].shift(1)>stock_df['predicted_price']) & (abs((stock_df['predicted_price']-stock_df['true_price'].shift(1))/stock_df['true_price'].shift(1)) >= threshold)
stock_df['long_condition'] = stock_df['long_condition'].astype(int)
stock_df['short_condition'] = stock_df['short_condition'].astype(int)

stock_df

,股票代號,日期,low,close,change,open,high,capacity,upper_bb,ma5,...,foreign_sell,foreign_buy,five_day_min_low,five_day_max_high,predicted_price,true_price,predicted_price_up,actual_price_up,long_condition,short_condition
日期,,,,,,,,,,,,,,,,,,,,,
2022-01-11,1213,2022-01-11,7.83,8.49,0.19,7.83,8.49,13,8.58,8.342,...,0.0,0.0,7.83,9.00,0.207943,0.212027,0,0,0,0
2022-01-12,1213,2022-01-12,8.24,8.38,-0.11,8.24,8.41,17,8.58,8.330,...,0.0,0.0,7.83,9.00,0.205185,0.205208,0,0,0,1
2022-01-13,1213,2022-01-13,8.21,8.21,-0.17,8.35,8.40,31,8.57,8.324,...,0.0,0.0,7.83,9.00,0.204401,0.194668,0,0,0,1
2022-01-14,1213,2022-01-14,8.23,8.25,0.04,8.34,8.40,11,8.57,8.326,...,0.0,0.0,7.83,9.00,0.204538,0.197148,1,1,1,0
2022-01-17,1213,2022-01-17,8.39,8.39,0.14,8.39,8.39,1,8.54,8.344,...,0.0,0.0,7.83,8.49,0.204463,0.205828,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,1213,2022-12-26,7.01,7.02,0.05,7.01,7.10,8,7.47,7.016,...,0.0,0.0,6.92,7.50,0.130664,0.120893,1,1,1,0
2022-12-27,1213,2022-12-27,6.83,6.83,-0.19,7.25,7.37,4,7.48,6.976,...,0.0,0.0,6.83,7.37,0.125990,0.109114,0,0,1,0
2022-12-28,1213,2022-12-28,6.63,6.86,0.03,6.63,6.87,8,7.47,6.942,...,0.0,0.0,6.63,7.37,0.125543,0.110973,0,1,1,0


strategy


In [2]:
# long strategy
signals_long = np.zeros(len(stock_df))
partition_size_long = 0
stop_loss_long = 0
for i in range(len(stock_df)):
    if partition_size_long == 0:
        if stock_df['long_condition'][i] == 1:
            signals_long[i] = 1
            partition_size_long = 1
            stop_loss_long = stock_df['five_day_min_low'][i]
    if partition_size_long == 1:
        if (stock_df['close'][i] <= stop_loss_long) or (stock_df['short_condition'][i] == 1): # 停利停損
            signals_long[i] = -1
            stop_loss_long = 0 # 停損點重置
            partition_size_long = 0 # 平倉

# short strategy
signals_short = np.zeros(len(stock_df))
partition_size_short = 0
stop_loss_short = 0
for i in range(len(stock_df)):
    if partition_size_short == 0:
        if stock_df['short_condition'][i] == 1:
            signals_short[i] = -1
            partition_size_short = -1
            stop_loss_short = stock_df['five_day_max_high'][i]
    if partition_size_short == -1:
        if (stock_df['close'][i] >= stop_loss_short) or (stock_df['long_condition'][i] == 1):# 停利停損
            signals_short[i] = 1
            stop_loss_short = 0 # 停損點重置
            partition_size_short = 0 # 平倉

entries_long = signals_long == 1
exits_long = signals_long == -1
entries_short = signals_short == -1
exits_short = signals_short == 1
pf = vbt.Portfolio.from_signals(stock_df['open'], 
                                    entries=entries_long,
                                    exits=exits_long, 
                                    short_entries=entries_short,
                                    short_exits=exits_short,
                                    fees = 0,
                                    freq='1D',)
# print(pf_long.stats().to_string()) # to_string()可以將全部結果攤開
print(pf.stats().to_string())
'''entries_short = signals_short == -1
exits_short = signals_short == 1
pf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')'''
# print(pf_short.stats().to_string())

C:\Users\111030\AppData\Local\Temp\ipykernel_79496\2491479233.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if stock_df['long_condition'][i] == 1:
C:\Users\111030\AppData\Local\Temp\ipykernel_79496\2491479233.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  stop_loss_long = stock_df['five_day_min_low'][i]
C:\Users\111030\AppData\Local\Temp\ipykernel_79496\2491479233.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (

Start                               2022-01-11 00:00:00
End                                 2022-12-30 00:00:00
Period                                240 days 00:00:00
Start Value                                       100.0
End Value                                   1202.960123
Total Return [%]                            1102.960123
Benchmark Return [%]                          -8.812261
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              16.805721
Max Drawdown Duration                  22 days 00:00:00
Total Trades                                         97
Total Closed Trades                                  96
Total Open Trades                                     1
Open Trade PnL                                      0.0
Win Rate [%]                                  76.041667
Best Trade [%]                                10.946746
Worst Trade [%]                                -

"entries_short = signals_short == -1\nexits_short = signals_short == 1\npf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')"

In [3]:
pf.plot().show()